## Smart RAG
- Know what you know and what you don't know.


In [16]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [17]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
import warnings

warnings.filterwarnings("ignore")

In [19]:
solar_summary = """
SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling

We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

llm = ChatUpstage()


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."

    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [21]:
chain.invoke({"question": "What is DUS?", "context": solar_summary})

'The answer is: DUS stands for Depth Up-Scaling.'

In [22]:
chain.invoke({"question": "How to get to Seoul from SF", "context": solar_summary})

'The information is not present in the context.'

In [23]:
# RAG or Search?
def is_in(question, context):
    is_in_conetxt = """As a helpful assistant, 
please use your best judgment to determine if the answer to the question is within the given context. 
If the answer is present in the context, please respond with "yes". 
If not, please respond with "no". 
Only provide "yes" or "no" and avoid including any additional information. 
Please do your best. Here is the question and the context:
---
CONTEXT: {context}
---
QUESTION: {question}
---
OUTPUT (yes or no):"""

    is_in_prompt = PromptTemplate.from_template(is_in_conetxt)
    chain = is_in_prompt | ChatUpstage() | StrOutputParser()

    response = chain.invoke({"context": context, "question": question})
    print(response)
    return response.lower().startswith("yes")

In [24]:
is_in("How to get to Seoul from SF", solar_summary)

no


False

In [25]:
is_in("What is DUS?", solar_summary)

yes


True

In [26]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def smart_rag(question, context):
    if not is_in(question, context):
        print("Searching in tavily")
        tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
        context = tavily.search(query=question)

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [27]:
smart_rag("What is DUS?", solar_summary)

yes


'The answer to the question "What is DUS?" is:\n\nDepth Up-Scaling (DUS)'

In [28]:
smart_rag("How to get to Seoul from SF?", solar_summary)

no.
Searching in tavily


'The answer to "How to get to Seoul from SF?" is:\n\n1. Fly from San Francisco (SFO) to Seoul (ICN) with airlines such as ANA, Japan Airlines, Asiana Airlines, Korean Air, and United Airlines.\n2. Take a train from Incheon Int\'l Airport T1 to Seoul Station.\n3. Take the BART from Civic Center / UN Plaza to Milpitas and then fly from San Jose (SJC) to Incheon (ICN).\n\nPlease note that the cheapest flights from San Francisco to Seoul start at $453 with AIR PREMIA.'

# Excercise

The `is_in` function sometimes works, but other times it does not. You can significantly improve it by providing a more detailed description and adding two or three examples in the prompt to see how it performs better.